This notebook have 3 parts:

1. Read one SCV file. 
2. Process noise from CSV files. Use the code if some rondom noise get coupled in and you are intrested in the noise frequency.
3. Process DAQ data

### 1. Read CSV

In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt

filename = "DAQ_TAC.csv" #filename
TAC = []
with open(filename, newline='') as csvfile:
    dataset = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in dataset:
        txt = ', '.join(row).split(",")
        if txt[0] == '':
            data_x = float(txt[-1]) #Choose the column you want to read
            TAC.append(data_x)
TAC = np.array(TAC)

In [ ]:
plt.hist(TAC,range=(5e-3,5e-2),bins = 100)
#plt.yscale("log")
plt.xlabel("TAC amplitude (V)")
plt.ylabel("Counts")
plt.title("Plot #1")
plt.show()

### 2. Use pyCRP to calculate noise

#### input csv files
Take 200 to 400 traces with the oscilloscope

In [ ]:
import pycrp

import csv
import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy
import joblib

def process_noise(filename, trace_length = 10000, channel = 1, fs = 1/(2.9992e-06 - 2.999E-06), cut_noise = False):
    files = glob.glob(filename)
    
    noise = [] #empty list for traces

    for file in files: # Loop over all the traces
        # For one trace
        with open(file, newline='') as csvfile:
            data_file = csv.reader(csvfile, delimiter=' ', quotechar='|')
            trace_3 = []
            for row in data_file: #loop over the file to pull the traces 
                txt = ', '.join(row).split(",")
                if txt[0] == '':
                    data_x = txt[3:-1]

                    trace_3.append(float(data_x[channel]))


            noise.append(trace_3[0:trace_length])
    print("noise:",len(noise))
    print("tracelength:",len(trace_3))

    noise=np.array(noise)

    noise = {"CH1":noise}

    utrace=pycrp.Trace(noise,fs=fs)
    if cut_noise:
        traces_cut = utrace.autocut_psd(max_iter=8,ds=4,use_filter="gauss:30,1", verbose=True,simultaneous_cut=False)
        unoise=pycrp.Noise(traces_cut,fs=fs,trace_weight=1e6,max_traces = 500)
    else:
        unoise=pycrp.Noise(noise,fs=fs,trace_weight=1e6,max_traces = 500)
    unoise.calc_psd()
    
    #PSDs_with_wire={"f":unoise.f,"PSDs":unoise.psds}
    return {"f":unoise.f,"PSDs":unoise.psds}

def pulse_template(filename, channel = 1,trace_length=2048, fs = 1/(2.9992e-06 - 2.999E-06), pre_trig = 1024, post_trig = 1024):
    CH3 = []
    trigger_points_CH3 = []
    files = glob.glob(filename)
    
    noise = [] #empty list for traces

    for file in files: # Loop over all the traces
        # For one trace
        with open(file, newline='') as csvfile:
            data_file = csv.reader(csvfile, delimiter=' ', quotechar='|')
            trace_3 = []
            for row in data_file: #loop over the file to pull the traces 
                txt = ', '.join(row).split(",")
                if txt[0] == '':
                    data_x = txt[3:-1]

                    trace_3.append(float(data_x[channel]))
            trace_3_filtered = scipy.ndimage.gaussian_filter(trace_3,sigma=10, order=0)
            trigger_pt_3 = np.argmin(trace_3_filtered) # This is not a rigorous method, but usually enough

        #Make sure the trace length is correct, which means the trigger point is not on the edge of the trace
        if len(trace_3[(trigger_pt_3-pre_trig):(trigger_pt_3+post_trig)]) == trace_length: 
            CH3.append(trace_3[(trigger_pt_3-pre_trig):(trigger_pt_3+post_trig)])
            trigger_points_CH3.append(trigger_pt_3)
    print("Number of traces averaged",len(np.array(CH3)))
    
    return np.average(np.array(CH3),axis = 0)

Template(pulse average) vs. noise\
Find the cutoff frequency from the plot

In [ ]:
filename = "sipm_dark_current_*/*.csv"
pulse_average = pulse_template(filename, channel = 1,trace_length=2048, fs = 1/(2.9992e-06 - 2.999E-06), pre_trig = 1024, post_trig = 1024) 
# fs is the scope sampling frequency. Find out from the scv file using 1/(t1-t0)
mean_0 = pulse_average-np.mean(pulse_average[0:1000])
pulse_average_normalize = -mean_0/np.min(mean_0)

noise = process_noise(filename, trace_length = 2048, channel = 1, fs = 1/(2.492e-7 - 2.49E-7), cut_noise = True)
plt.figure(figsize=(12,8))
plt.plot(noise["f"]/1e6, noise["PSDs"]["CH1"], label='noise',alpha=0.6)
freq, power = scipy.signal.periodogram(pulse_average*1e6, fs=1/(2.492e-7 - 2.49E-7))
plt.plot(freq/1e6, power, label='pulse',alpha = 0.6)

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Power (uV/sqrt(Hz))')
plt.ylim(1e-7, 0.5)
plt.title("Noise PSDs")
plt.legend()

In [ ]:
def get_trace_from_csv(filename, channel = 1,trace_length=2048, fs = 1/(0.02e-7),pre_trig = 1024, post_trig = 1024):
    CH3 = []
    trigger_points_CH3 = []
    files = glob.glob(filename)
    
    noise = [] #empty list for traces

    for file in files: # Loop over all the traces
        # For one trace
        with open(file, newline='') as csvfile:
            data_file = csv.reader(csvfile, delimiter=' ', quotechar='|')
            trace_3 = []
            for row in data_file: #loop over the file to pull the traces 
                txt = ', '.join(row).split(",")
                if txt[0] == '':
                    data_x = txt[3:-1]

                    trace_3.append(float(data_x[channel]))
            trace_3_filtered = scipy.ndimage.gaussian_filter(trace_3,sigma=10, order=0)
            print(len(trace_3_filtered))
            trigger_pt_3 = np.argmin(trace_3_filtered)
            #trigger_pt_3 = 1248


        if len(trace_3[(trigger_pt_3-pre_trig):(trigger_pt_3+post_trig)]) == trace_length: #Make sure the trace length is correct.
            CH3.append(trace_3[(trigger_pt_3-pre_trig):(trigger_pt_3+post_trig)])
            trigger_points_CH3.append(trigger_pt_3)
    
    return np.array(CH3)

traces = get_trace_from_csv(filename, channel = 2,trace_length=1024, fs = 1/(0.02e-7),pre_trig = 512, post_trig = 512)
#trace_length must match the noise trace length

#### Find amplitude

In [ ]:
result = []
OFL_amplitude = []
OFL_chi2 = []
OFL_shift = []
OF0_amplitude = []
OF0_chi2 = []
for data in traces:
    fs = 5e9 # Scope sampling frequency
    OF = pycrp.OptimalFilter(fs = Fs)
    OFresult = OF.of_amp(S=data,N=noise["PSDs"]["CH1"],T=pulse_average_normalize,fs=Fs,delay=25,
                         LPF=1e8,LPF_OFP=10e8,interpolate=False,return_all=True,
                         OFP_length=-1,OFP_threshold=0)
    OF0_amplitude.append(OFresult[0][0])
    OF0_chi2.append(OFresult[0][1])    
    OFL_amplitude.append(OFresult[2][0])
    OFL_chi2.append(OFresult[2][1])
    OFL_shift.append(OFresult[1][2])
    
RQs = {"OF0":np.array(OF0_amplitude), "OF0_chi2":np.array(OF0_chi2),
       "OFL":np.array(OFL_amplitude), "OFL_chi2":np.array(OFL_chi2),"OFL_time":np.array(OFL_shift), }

In [ ]:
mask = (RQs["OFL_chi2"] < 3)
histgram = plt.hist(RQs["OFL"][mask]*1e3,range = (-2, 16),bins = 200)
plt.xlabel("OFL amplitude(mV)")
plt.ylabel("counts")
first_peak_fit = pycrp.Fit.fitGaus_from_hist(histgram[0], histgram[1], fitrange=(1,6), poissonerror=False,
                  interp_points=100, p0=None, make_plot=True, sidebands=False, label = "")
second_peak_fit = pycrp.Fit.fitGaus_from_hist(histgram[0], histgram[1], fitrange=(5,10), poissonerror=False,
                  interp_points=100, p0=None, make_plot=True, sidebands=False, label = "")
third_peak_fit = pycrp.Fit.fitGaus_from_hist(histgram[0], histgram[1], fitrange=(8,13), poissonerror=False,
                  interp_points=100, p0=None, make_plot=True, sidebands=False, label = "")
plt.title("Bandgap of Boardcom SiPM")
plt.legend()

### 3. DAQ data processing

In [ ]:
def find_channel(filename, noise_reject_rate = 0.01):
    ch_avalible =[]
    which_channels = []
    for i in range(32):
        ch_avalible.append("HIT_0_{0}".format(i))
        ch_avalible.append("HIT_1_{0}".format(i))
    hit_df = pd.read_csv(filename, sep=';', on_bad_lines='skip', usecols=ch_avalible).fillna(0).to_numpy()
    ch_list = np.argwhere(np.sum(hit_df, axis = 0) > noise_reject_rate*len(hit_df))
    which_channels = []
    for index in ch_list:
        if index[0] >= 32:
            which_channels.append(index[0]-32)
        else:
            which_channels.append(index[0])
    return which_channels

def load_dataset(filename, which_channels = ["0_8","0_20"], noise_reject_rate = 0.01):
    """
    Load DT5550W DAQ dataset (processing mode = "FULL" in the software)
    
    Inputs
    -------
    filename: str
        Filename path
    which_channels: list or "auto"
        If which_channels is a list, read the sepcified channels
        If which_channels is "auto", find active channels 
    noise_reject_rate: float in [0,1]
        The percentage of events needed to be identified as active channels, required only when which_channels is "auto"
        
    Returns
    -------
    df: Pandas dataframe
    """
    cols_list = pd.read_csv(filename, nrows=0).columns.tolist()[0].split(sep=";")
    #metadata_name = cols_list[:10]+cols_list[int(len(cols_list)/2+2):int(len(cols_list)/2+8)]
    metadata_name = cols_list[:10]
    dtype_list = ["HIT_","CHARGE_", "COARSE_", "FINE_","RELATIVETIME_"]
    #dtype_list = ["HIT_","CHARGE_", "COARSE_", "FINE_",]
    if which_channels == "auto":
        ch_hit =[]
        for i in range(32):
            ch_hit.append("HIT_0_{0}".format(i))
            ch_hit.append("HIT_1_{0}".format(i))
        hit_df = pd.read_csv(filename, sep=';', on_bad_lines='skip', usecols=ch_hit).fillna(0).to_numpy()
        
        ch_list = np.argwhere(np.sum(hit_df, axis = 0) > noise_reject_rate*len(hit_df))
        which_channels = []
        for index in ch_list:
            if index >= 32:
                which_channels.append("1_{0}".format(index[0]-32))
            else:
                which_channels.append("0_{0}".format(index[0]))
    data_ch = []
    for channels in which_channels:
        for ch_name in dtype_list:
            data_ch.append(ch_name+channels)
    return pd.read_csv(filename, sep=';', on_bad_lines='skip', usecols=metadata_name+data_ch)


def find_alpha(df, channels = ['FINE_0_8', 'FINE_0_26']):
    """
    FInd the calibration parameters for channels
    
    Inputs
    -------
    df: Dict
        A dictionary from Pandas dataframe for DT5550W DAQ dataset

    channels: list
        Channels to calibrate
    
    Returns
    -------
    alpha: list
        calibration constent for channels
    """
    
    alpha = np.zeros(len(channels))
    for i,ch in enumerate(channels):
        mask = (np.array(df[ch]) > 4)& (np.array(df[ch]) < 1020) #remove fine time = 4, 1020 events
        #mask = (np.array(df[ch]) > 200)& (np.array(df[ch]) < 1020) 
        print("Fine time TAC min:", np.min(np.array(df[ch])[mask]))
        print("Fine time TAC max:", np.max(np.array(df[ch])[mask]))
        alpha[i] = 25/(np.max(np.array(df[ch])[mask])-np.min(np.array(df[ch])[mask]))
    return alpha



In [ ]:
def trigger(filename, trigger_ch = "0_4"):
    """
    Use one channel as the trigger channel
    """
    df = load_dataset(filename, which_channels = "auto", noise_reject_rate = 0.05)
    trigger_ch = 'HIT_'+trigger_ch
    return df.where((df[trigger_ch] == 1)&(df[ch] == 1)).dropna()

#Example:
#trigger(filename, trigger_ch = "0_4")

An example for ToF related plots

In [ ]:
# ch1 = 4
# ch2 = 26

#plt.figure(figsize=(12, 8), dpi=100)
#txt = ["16*2 ns LEMO on CH26", "16ns LEMO on CH26", "No LEMO delay", "16ns LEMO on CH4","16*2 ns LEMO on CH4","16 ns LEMO on CH4","No LEMO delay"]

for i in range(4):
    filename = "0330/{0}.data".format(i+1)
    try:
        ch1, ch2 = find_channel(filename, noise_reject_rate = 0.15)[-2:]
        df = load_dataset(filename, which_channels = "auto", noise_reject_rate = 0.15).to_dict('list')
        print(df.keys())

        #remove fine time = 4, 1020 events
        mask = ((np.array(df['FINE_0_{0}'.format(ch1)]) > 4)&(np.array(df['FINE_0_{0}'.format(ch1)]) < 1020)&
                (np.array(df['FINE_0_{0}'.format(ch2)]) > 4)&(np.array(df['FINE_0_{0}'.format(ch2)]) < 1020)) 
        
        print("Number of hits on the 1st ch:",np.array(df['HIT_0_{0}'.format(ch1)]).sum())
        print("Number of hits on the 2nd ch:",np.array(df['HIT_0_{0}'.format(ch2)]).sum())

        alpha = find_alpha(df, channels = ['FINE_0_{0}'.format(ch1), 'FINE_0_{0}'.format(ch2)])
        fine_time = np.multiply(alpha,np.array([df['FINE_0_{0}'.format(ch1)]-np.min(np.array(df['FINE_0_{0}'.format(ch1)])[mask]),
                                                df['FINE_0_{0}'.format(ch2)]-np.min(np.array(df['FINE_0_{0}'.format(ch2)])[mask])]).T)
        #print(fine_time)
        coarse_time = np.array([df['COARSE_0_{0}'.format(ch1)],df['COARSE_0_{0}'.format(ch2)]])*25

        time = (coarse_time-fine_time.T)
        ToF = time[0]-time[1]

        plt.figure(figsize=(12, 8), dpi=100)

        # plot fine time
        
#         plt.hist(fine_time.T[0],bins = 400, range = (-1,26), histtype = "step")
#         plt.xlabel("Fine time(ns) ASIC-A CH{0}".format(ch1))
#         plt.ylabel("counts")
#         plt.title("Fine time of one channel")
        
        # plot fine time 2d hist

#         plt.figure(figsize=(12, 8), dpi=100)
#         plt.hist2d(ToF, fine_time.T[1] ,bins=(200,50), range=[[-20,60],[-1,26]])
#         plt.xlabel("ToF(ns) CH8 - CH18")
#         plt.ylabel("Fine time(ns) CH18")
        
        # plot ToF

        center = np.abs(np.median(ToF))
        
        print("number of events:",count(ToF, center))
        
        hist1 = plt.hist(np.abs(ToF),bins = 50, range = (center-10,center+10), histtype = "step")
        fit = pycrp.Fit()
        #fit_1 = fit.fitGaus_from_hist(hist1[0], hist1[1], interp_points=200, make_plot=True,label = "{0}".format(txt[i]))
        fit_1 = fit.fitGaus_from_hist(hist1[0], hist1[1], interp_points=200, make_plot=True)
        print(fit_1[0][1], fit_1[0][2])
        plt.xlabel("ToF(ns) CH{0} - CH{1}".format(ch1,ch2))
        plt.ylabel("counts")
        #plt.title("ToF measurement with different channels on ASIC_B")
        plt.show()
    except:
        print(find_channel(filename, noise_reject_rate = 0.2))


print dataset:

In [ ]:
filename = "0323/2.data"
load_dataset(filename, which_channels = ["0_5","0_19"], noise_reject_rate = 0.05)